<a href="https://colab.research.google.com/github/simoncamal/tutorial-forecasting-trading/blob/main/previ_extremes_gefcom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import plotly.express as px
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
from scipy import signal
from sklearn.linear_model import LassoCV, Lasso

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 39.9 MB/s eta 0:00:00


In [ ]:

!wget https://raw.githubusercontent.com/simoncamal/tutorial-forecasting-trading/main/tutorial_data/dataset_gefcom_prod_logit.pkl
!wget https://raw.githubusercontent.com/simoncamal/tutorial-forecasting-trading/main/tutorial_data/dataset_gefcom_wd.pkl


--2023-11-25 14:07:30--  https://raw.githubusercontent.com/simoncamal/tutorial-forecasting-trading/main/tutorial_data/dataset_gefcom_prod_logit.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1534698 (1.5M) [application/octet-stream]
Saving to: ‘dataset_gefcom_prod_logit.pkl’

dataset_gefcom_prod 100%[===================>]   1.46M  --.-KB/s    in 0.04s   

2023-11-25 14:07:31 (34.5 MB/s) - ‘dataset_gefcom_prod_logit.pkl’ saved [1534698/1534698]

--2023-11-25 14:07:31--  https://raw.githubusercontent.com/simoncamal/tutorial-forecasting-trading/main/tutorial_data/dataset_gefcom_wd.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubu

Definition training testing



In [ ]:
dataset_prod = pd.read_pickle('/content/dataset_gefcom_prod_logit.pkl')
dataset_prod = dataset_prod.set_index('TIMESTAMP')
dataset_wd = pd.read_pickle('/content/dataset_gefcom_wd.pkl')

conditioning_number_points = 10
conditioning_winddirection_points_positions = np.arange(0, 360, 36)
conditioning_winddirection_bandwidths = [90]
number_plants = 10
number_horizons = 6
lags = [0, 1, 2, 4, 6]
alphas_lasso = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]

target_site = dataset_wd.columns[0]
testing_month_index = 1




1. Aux functions: dataset preprocessing

In [ ]:
def train_test_splits_windDirection(dataset_wd, testing_month_index):
  wd_train_base = dataset_wd[dataset_wd.index.year == 2012]
  wd_test_month = dataset_wd[(dataset_wd.index.year == 2013)&(dataset_wd.index.month == testing_month_index)]
  wd_train = dataset_wd[dataset_wd.index < wd_test_month.index[0]]

  return wd_train, wd_test_month


def train_test_splits_Production(dataset_prod, testing_month_index):
  prod_train_base = dataset_prod[dataset_prod.index.year == 2012]
  prod_test_month = dataset_prod[(dataset_prod.index.year == 2013)&(dataset_prod.index.month == testing_month_index)]
  prod_train = dataset_prod[dataset_prod.index < prod_test_month.index[0]]

  return prod_train, prod_test_month

cor prod

In [ ]:
prod_train = dataset_prod[dataset_prod.index.year == 2012]
cor_prod = []
for target_site in np.arange(1, 11, 1):
  cor_prod_target = pd.DataFrame({'cor':
    np.asarray([np.correlate(prod_train[str(target_site)].values,
                            prod_train[col].values)/(np.std(prod_train[str(target_site)].values)*np.std(prod_train[col].values)*prod_train.shape[0])
                            for col in prod_train.columns]).flatten(),
                                  'site_i': target_site, 'site_j':prod_train.columns})
  cor_prod.append(cor_prod_target)
cor_prod = pd.concat(cor_prod)
print(cor_prod)

         cor  site_i site_j
0   1.412208       1      1
1   0.816240       1      2
2   0.651848       1      3
3   0.722273       1      4
4   0.562570       1      5
..       ...     ...    ...
5   0.697741      10      6
6   0.427707      10      7
7   0.395371      10      8
8   0.472078      10      9
9   1.042543      10     10

[100 rows x 3 columns]


ccf wd

In [ ]:
ccf_target_train = np.asarray([np.correlate(wd_train[target_site].values, wd_train[col].values)/(np.std(wd_train[target_site].values)*np.std(wd_train[col].values)*wd_train.shape[0]) for col in wd_train.columns]).flatten()
ccf_target_test = pd.DataFrame({'ccf_wd':
  np.asarray([np.correlate(wd_test_month[target_site].values, wd_test_month[col].values)/(np.std(wd_test_month[target_site].values)*np.std(wd_test_month[col].values)*wd_test_month.shape[0]) for col in wd_test_month.columns]).flatten(),
                                'site_i': target_site, 'site_j':wd_train.columns})
print(ccf_target_test.sort_values(by='ccf_wd', ascending=False))

NameError: ignored

In [ ]:
# Lasso conditioned by wind direction


def buildExplanVar(dataset, lags, horizon, number_plants, explanatory_sites):
    explan_vars = dataset[[str(s) for s in explanatory_sites]]
    explan_vars_lagged = explan_vars.assign(
        **{f'{col} (t-{lag + horizon})': explan_vars[col].shift(lag + horizon).fillna(method='bfill')
           for lag in lags for col in explan_vars})
    explan_vars_lagged = explan_vars_lagged.iloc[:, number_plants:]  # exclude observed at time t
    explan_vars_lagged = explan_vars_lagged.assign(intercept=1)

    return explan_vars_lagged


def buildARST(dataset, explanatory_sites,
              nb_plants, lags, horizon, target_site, plot=False):

    explan_vars_lagged = buildExplanVar(dataset=dataset, explanatory_sites=explanatory_sites, lags=lags,
                                        horizon=horizon,
                                        number_plants=nb_plants)
    explan_vars_lagged_mat = explan_vars_lagged.values
    explan_vars_lagged_mat_t = np.transpose(explan_vars_lagged_mat)
    explan_covariance_matrix = np.matmul(explan_vars_lagged_mat_t, explan_vars_lagged_mat)
    norm_factor = np.max(explan_covariance_matrix)
    explan_covariance_matrix = explan_covariance_matrix / norm_factor

    explan_precision_matrix = linalg.inv(explan_covariance_matrix)
    xty_matrix = np.matmul(explan_vars_lagged_mat_t, dataset[str(target_site)].values) / norm_factor
    response = dataset[str(target_site)].values / norm_factor
    beta_ar_noweights = np.matmul(explan_precision_matrix, xty_matrix)
    beta_ar_noweights_df = pd.DataFrame({'beta': beta_ar_noweights,
                                         'explanatory_site': np.append(
                                             np.concatenate([np.repeat(site, len(lags)) for site in explanatory_sites]),
                                             'intercept'),
                                         'explanatory_lag': np.append(
                                             np.concatenate([[lag + horizon for lag in lags] for i in range(nb_plants)]),
                                             'NA')})
    beta_ar_noweights_wide_df = beta_ar_noweights_df.pivot_table(values='beta', columns='explanatory_lag',
                                                                 index='explanatory_site')
    beta_ar_noweights_wide_df[np.isnan(beta_ar_noweights_wide_df)] = 0

    if plot:
        white_scale_point = 0 - beta_ar_noweights_df['beta'].min() / (
                beta_ar_noweights_df['beta'].max() - beta_ar_noweights_df['beta'].min())
        red_white_green_scale = [[0, 'rgba(214, 39, 40, 0.85)'],
                                 [white_scale_point, 'rgba(255, 255, 255, 0.85)'],
                                 [1, 'rgba(6,54,21, 0.85)']]
        px.imshow(beta_ar_noweights_wide_df, color_continuous_scale=red_white_green_scale).show()

    return beta_ar_noweights_df, beta_ar_noweights_wide_df, explan_covariance_matrix, xty_matrix, response

def buildAR(dataset, horizon, lags, target_site, plot=False):
    explan_vars_lagged = buildExplanVar(dataset=dataset, explanatory_sites=[target_site], lags=lags, horizon=horizon,
                                        number_plants=1)
    explan_vars_lagged_mat = explan_vars_lagged.values
    explan_vars_lagged_mat_t = np.transpose(explan_vars_lagged_mat)
    explan_covariance_matrix = np.matmul(explan_vars_lagged_mat_t, explan_vars_lagged_mat) / (
        explan_vars_lagged.shape[0])
    explan_precision_matrix = linalg.inv(explan_covariance_matrix)
    xty_matrix = np.matmul(explan_vars_lagged_mat_t, dataset[str(target_site)].values) / \
                 explan_vars_lagged.shape[0]
    beta_ar_noweights = np.matmul(explan_precision_matrix, xty_matrix)
    beta_ar_noweights_df = pd.DataFrame({'beta': beta_ar_noweights,
                                         'explanatory_site': np.append(np.repeat(target_site, len(lags)),
                                                                       'intercept'),
                                         'explanatory_lag': np.append([lag + horizon for lag in lags], 'NA')})
    beta_ar_noweights_wide_df = beta_ar_noweights_df.pivot_table(values='beta', columns='explanatory_lag',
                                                                 index='explanatory_site')
    # beta_ar_noweights_df.assign(feature=[beta_ar_noweights_df.iloc[i, :]['explanatory_site']+'_lag_'+beta_ar_noweights_df.iloc[i,:]['explanatory_lag'] for i in range(beta_ar_noweights_df.shape[0])])
    beta_ar_noweights_wide_df[np.isnan(beta_ar_noweights_wide_df)] = 0

    if plot:
        white_scale_point = 0 - beta_ar_noweights_df['beta'].min() / (
                beta_ar_noweights_df['beta'].max() - beta_ar_noweights_df['beta'].min())
        red_white_green_scale = [[0, 'rgba(214, 39, 40, 0.85)'],
                                 [white_scale_point, 'rgba(255, 255, 255, 0.85)'],
                                 [1, 'rgba(6,54,21, 0.85)']]
        px.imshow(beta_ar_noweights_wide_df, color_continuous_scale=red_white_green_scale).show()

    return beta_ar_noweights_df, beta_ar_noweights_wide_df


def buildARweighted(dataset, weights, conditioning_points, conditioning_bandwidth,
                    horizon, lags, target_site, plot=False):
    explan_vars_lagged = buildExplanVar(dataset=dataset, explanatory_sites=[target_site], lags=lags, horizon=horizon,
                                        number_plants=1)
    explan_vars_lagged_mat = explan_vars_lagged.values
    explan_vars_lagged_mat_t = np.transpose(explan_vars_lagged_mat)

    beta_dfs = []
    beta_wide_dfs = []
    for i in range(len(weights)):
        weights_col = weights[i]
        conditioning_point = conditioning_points[i]
        explan_weighted_covariance_matrix = np.matmul(explan_vars_lagged_mat_t * weights_col,
                                                      explan_vars_lagged_mat) / (explan_vars_lagged.shape[0])
        explan_precision_matrix = linalg.inv(explan_weighted_covariance_matrix)
        xtwy_matrix = np.matmul(explan_vars_lagged_mat_t * weights_col, dataset[target_site].values) / \
                      explan_vars_lagged.shape[0]
        beta = np.matmul(explan_precision_matrix, xtwy_matrix)
        beta_df = pd.DataFrame({'beta': beta,
                                'conditioning_point': conditioning_point,
                                'conditioning_bandwidth': conditioning_bandwidth,
                                'explanatory_site': np.append(np.repeat(target_site, len(lags)),
                                                              'intercept'),
                                'explanatory_lag': np.append([lag + h for lag in lags], 'NA')})
        beta_wide_df = beta_df.pivot_table(values='beta', columns='explanatory_lag',
                                           index=['explanatory_site', 'conditioning_point', 'conditioning_bandwidth'])
        beta_wide_df[np.isnan(beta_wide_df)] = 0
        beta_dfs.append(beta_df)
        beta_wide_dfs.append(beta_wide_df)
    beta_df = pd.concat(beta_dfs)
    beta_wide_df = pd.concat(beta_wide_dfs)

    if plot:
        white_scale_point = 0 - beta_df['beta'].min() / (
                beta_df['beta'].max() - beta_df['beta'].min())
        red_white_green_scale = [[0, 'rgba(214, 39, 40, 0.85)'],
                                 [white_scale_point, 'rgba(255, 255, 255, 0.85)'],
                                 [1, 'rgba(6,54,21, 0.85)']]
        px.imshow(beta_wide_df, color_continuous_scale=red_white_green_scale).show()

    return beta_df, beta_wide_df

def buildARSTweighted(dataset, weights, explanatory_sites, conditioning_points, conditioning_bandwidth,
                    horizon, lags, target_site, number_plants, plot=False):
    explan_vars_lagged = buildExplanVar(dataset=dataset, explanatory_sites=explanatory_sites, lags=lags, horizon=horizon,
                                        number_plants=number_plants)
    explan_vars_lagged_mat = explan_vars_lagged.values
    explan_vars_lagged_mat_t = np.transpose(explan_vars_lagged_mat)

    beta_dfs = []
    beta_wide_dfs = []
    xtx_w_list = []
    xty_w_list = []
    response_list = []
    for i in range(len(weights)):
        weights_col = weights[i]
        conditioning_point = conditioning_points[i]
        explan_weighted_covariance_matrix = np.matmul(explan_vars_lagged_mat_t * weights_col,
                                                      explan_vars_lagged_mat)
        norm_factor = np.max(explan_weighted_covariance_matrix)
        explan_weighted_covariance_matrix = explan_weighted_covariance_matrix / norm_factor
        xtx_w_list.append(explan_weighted_covariance_matrix)
        explan_precision_matrix = linalg.inv(explan_weighted_covariance_matrix)
        xtwy_matrix = np.matmul(explan_vars_lagged_mat_t * weights_col, dataset[str(target_site)].values) / norm_factor
        xty_w_list.append(xtwy_matrix)
        response_list.append(dataset[str(target_site)].values / norm_factor)

        beta = np.matmul(explan_precision_matrix, xtwy_matrix)
        beta_df = pd.DataFrame({'beta': beta,
                                'conditioning_point': conditioning_point,
                                'conditioning_bandwidth': conditioning_bandwidth,
                                'explanatory_site': np.append(
                                    np.concatenate([np.repeat(site, len(lags)) for site in explanatory_sites]),
                                    'intercept'),
                                'explanatory_lag': np.append(
                                    np.concatenate([[lag + h for lag in lags] for i in range(number_plants)]),'NA')})
        beta_wide_df = beta_df.pivot_table(values='beta', columns='explanatory_lag',
                                           index=['explanatory_site', 'conditioning_point', 'conditioning_bandwidth'])
        beta_wide_df[np.isnan(beta_wide_df)] = 0
        beta_dfs.append(beta_df)
        beta_wide_dfs.append(beta_wide_df)
    beta_df = pd.concat(beta_dfs)
    beta_wide_df = pd.concat(beta_wide_dfs)

    if plot:
        white_scale_point = 0 - beta_df['beta'].min() / (
                beta_df['beta'].max() - beta_df['beta'].min())
        red_white_green_scale = [[0, 'rgba(214, 39, 40, 0.85)'],
                                 [white_scale_point, 'rgba(255, 255, 255, 0.85)'],
                                 [1, 'rgba(6,54,21, 0.85)']]
        px.imshow(beta_wide_df, color_continuous_scale=red_white_green_scale).show()

    return beta_df, beta_wide_df, xtx_w_list, xty_w_list, response_list


def buildPrediction(betas, dataset, target_site, explanatory_sites, horizon, lags, number_plants, method_name, **kwargs):
    weights = None
    plot = None
    logit_bool = False
    for key, value in kwargs.items():
        if key == 'conditioning_bandwidth':
            conditioning_bandwidth = value
        if key == 'weights':
            weights = value
        if key == 'conditioning_points':
            conditioning_points = value
        if key == 'plot':
            plot = value
        if key == 'logit_bool':
            logit_bool = value

    explan_vars_lagged_testing = buildExplanVar(dataset=dataset, lags=lags, horizon=horizon,
                                                number_plants=number_plants, explanatory_sites=explanatory_sites)

    prediction = np.zeros(explan_vars_lagged_testing.shape[0])
    if weights is None:

        #for column_index in range(explan_vars_lagged_testing.shape[1]):
            # print(column_index)
        #    feature_index = explan_vars_lagged_testing.columns[column_index]
        #    if ' ' in feature_index:
        #        explan_site, explan_site_lag = feature_index.split(' ')
        #        explan_site_lag = re.findall(r'\d+', explan_site_lag)[0]

        #    else:
        #        explan_site = feature_index
        #        explan_site_lag = 'NA'
        #    beta_column_index = \
        #    betas[(betas['explanatory_site'] == explan_site) & (betas['explanatory_lag'] == explan_site_lag)][
        #        'beta'].values[0]
        #    prediction = prediction + beta_column_index * explan_vars_lagged_testing[feature_index]

        prediction = np.matmul(betas['beta'].values, explan_vars_lagged_testing.T)
    else:

        sum_weights = weights.sum(axis=1).values # sum of weights for each sample

        #for weight_index in range(weights.shape[1]):
        #    weight = weights.iloc[:,weight_index]
        #    conditioning_point = conditioning_points[weight_index]
        #    for column_index in range(explan_vars_lagged_testing.shape[1]):
                # print(column_index)
        #        feature_index = explan_vars_lagged_testing.columns[column_index]
        #        if ' ' in feature_index:
        #            explan_site, explan_site_lag = feature_index.split(' ')
        #            explan_site_lag = re.findall(r'\d+', explan_site_lag)[0]
        #        else:
        #            explan_site = feature_index
        #            explan_site_lag = 'NA'
        #        beta_column_index = betas[(betas['explanatory_site'] == explan_site) &
        #                                  (betas['explanatory_lag'] == explan_site_lag) &
        #                                  (betas['conditioning_point'] == conditioning_point)][
        #            'beta'].values[0]
        #        pred_weight = beta_column_index * weight / sum_weights * explan_vars_lagged_testing[feature_index].values
        #        prediction = prediction + pred_weight

        for weight_index in range(weights.shape[1]):
          weight = weights.iloc[:,weight_index]
          conditioning_point = conditioning_points[weight_index]
          beta_column_index = betas[betas['conditioning_point'] == conditioning_point]['beta'].values
          prediction_weight = np.matmul(beta_column_index, explan_vars_lagged_testing.T)
          prediction = prediction + prediction_weight/sum_weights

    obs = dataset[str(target_site)]
    if logit_bool:
        obs = logit_inv(obs)
        prediction = logit_inv(prediction)
    prediction[prediction < 0] = 0
    prediction[prediction > 1] = 1
    prediction_df = pd.DataFrame(
        {'method': method_name, 'pred': prediction, 'obs': obs.values,
         'horizon': horizon, 'datetime': dataset.index.values,
         'target_site': str(target_site)})
    prediction_df['persistence_h'] = prediction_df['obs'].shift(horizon + 1).fillna(method='bfill')

    if plot:
        px.line(prediction_df.iloc[:2000,:].melt(id_vars=['datetime', 'horizon', 'method']),
            x='datetime', y='value', color='variable').show()

    score_tbl = pd.DataFrame({'rmse': [np.sqrt(np.mean((prediction_df['obs'] - prediction_df['pred']) ** 2))],
                              'mae': [np.mean(np.abs(prediction_df['obs'] - prediction_df['pred']))],
                              'rmse_persist': [
                                  np.sqrt(np.mean((prediction_df['obs'] - prediction_df['persistence_h']) ** 2))],
                              'mae_persist': [np.mean(np.abs(prediction_df['obs'] - prediction_df['persistence_h']))],
                              'horizon': [h],
                              'model': method_name})

    return score_tbl, prediction_df

def builARSTl1Solve_budget(XtX, XtY, l1_threshold, lasso, verbose):
    """
    Deploy and optimize the MIQP formulation of L1-Regression.
    """

    #assert isinstance(l1_threshold, (np.float))
    # Create a Gurobi environment and a model object for each conditioning weight

    with gp.Env(empty=True) as env:
        if not verbose:
            env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model("", env=env) as regressor:
            dim = XtY.shape[0]
            assert dim == XtX.shape[0]

            # Decision variables
            norm_1 = regressor.addVar(lb=l1_threshold, ub=l1_threshold, name="norm")
            beta = regressor.addMVar((dim,), lb=-GRB.INFINITY, name="beta") # Weights
            regressor.setObjective(beta.T @ XtX @ beta - 2 * XtY @ beta, GRB.MINIMIZE)

            # Budget constraint based on the L1-norm
            if lasso:
                regressor.addGenConstrNorm(norm_1, beta[:-1], which=1, name="budget") # respect l1 budget constraint, exclude intercept

            #regressor.params.timelimit = 60
            regressor.params.mipgap = 0.001
            regressor.optimize()

            coeff = np.array([beta[i].X for i in range(dim)])

        return coeff

def builARSTl1Solve_normconstraint(XtX, XtY, response, l1_threshold, verbose=True):
    """
    Deploy and optimize the MIQP formulation of L1-Regression.
    ONGOING
    """

    assert isinstance(l1_threshold, (np.float))
    # Create a Gurobi environment and a model object for each conditioning weight

    with gp.Env(empty=True) as env:
        if not verbose:
            env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model("", env=env) as regressor:
            dim = XtY.shape[0]
            assert dim == XtX.shape[0]

            # Decision variables
            #norm_1 = regressor.addVar(lb=0, ub=GRB.INFINITY, name="norm")
            beta = regressor.addMVar((dim,), lb=-GRB.INFINITY, name="beta") #Weights
            regressor.setObjective(beta.T @ XtX @ beta - 2 * XtY @ beta, GRB.MINIMIZE)

            # Inequality constraint based on the L1-norm
            #regressor.addConstr(norm_1 == gp.norm(beta[:-1], 1.0))
            #regressor.addConstr(norm_1 <= l1_threshold)

            #regressor.params.timelimit = 60
            regressor.params.mipgap = 0.001
            regressor.optimize()

            coeff = np.array([beta[i].X for i in range(dim)])

            return coeff

def builARSTl1Solve_normconstraint_vRSS(X, y, l1_threshold, verbose=True):
    """
    Deploy and optimize the MIQP formulation of L1-Regression.
    ONGOING
    """

    assert isinstance(l1_threshold, (np.float))
    # Create a Gurobi environment and a model object for each conditioning weight

    with gp.Env(empty=True) as env:
        if not verbose:
            env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model("", env=env) as regressor:

            Xval= X.values
            yval = y.values
            dim = X.shape[1]
            samples = X.shape[0]
            assert samples == y.shape[0]

            # scale features and response

            # Decision variables
            norm_1 = regressor.addVar(lb=0, ub=GRB.INFINITY, name="norm")
            beta = regressor.addMVar((dim,), lb=-GRB.INFINITY, name="beta") # Weights
            deviation = regressor.addMVar(samples, lb=-GRB.INFINITY, name='deviation')

            regressor.addConstr(deviation == y - Xval @ beta)
            loss = deviation @ deviation

            # Inequality constraint based on the L1-norm
            regressor.addConstr(norm_1 == gp.norm(beta[:-1], 1.0))
            regressor.addConstr(norm_1 <= l1_threshold)

            # Set objective
            regressor.setObjective(loss, GRB.MINIMIZE)

            #regressor.params.timelimit = 60
            regressor.params.mipgap = 0.001
            regressor.optimize()

            coeff = np.array([beta[i].X for i in range(dim)])

            return coeff


def builARSTl1Solve_normineq(XtX, XtY, l1_threshold, lasso=True, verbose=True):
    """
    Deploy and optimize the MIQP formulation of L1-Regression, with pos / neg beta decomposition.
    """

    #assert isinstance(l1_threshold, (np.float))
    # Create a Gurobi environment and a model object for each conditioning weight

    with gp.Env(empty=True) as env:
        if not verbose:
            env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model("", env=env) as regressor:
            dim = XtY.shape[0]
            assert dim == XtX.shape[0]

            # XtX extended to model absolute value
            xtx_extended = np.hstack((XtX, np.zeros((dim, dim))))
            xtx_extended = np.vstack((xtx_extended, np.zeros((dim, 2 * dim))))
            xtx_extended = np.vstack((xtx_extended, np.zeros((1, 2 * dim))))
            xtx_extended = np.hstack((xtx_extended, np.zeros((2 * dim + 1, 1))))
            xty_extended = np.hstack((XtY, np.zeros((dim + 1))))

            # Decision variables
            beta_extended = regressor.addMVar((2 * dim + 1,), lb=-GRB.INFINITY, name="beta") # Weights
            regressor.setObjective(beta_extended.T @ xtx_extended @ beta_extended - 2 * xty_extended @ beta_extended, GRB.MINIMIZE)

            # Expression of absolute norm inequality
            regressor.addConstr( -np.ones((1,dim)) @ beta_extended[np.arange(dim, 2 * dim , 1)] - dim * beta_extended[2 * dim] <= -l1_threshold)

            for i in range(dim):
                regressor.addConstr(- beta_extended[i] + beta_extended[dim + i] + beta_extended[2 * dim] <= 0)
                regressor.addConstr(beta_extended[i] + beta_extended[dim + i] + beta_extended[2 * dim] <= 0)

            #regressor.params.timelimit = 60
            regressor.params.mipgap = 0.001
            regressor.optimize()

            coeff = np.array([beta_extended[i].X for i in range(dim)])
            beta_x = np.array([beta_extended[dim -1 + i].X for i in range(dim)])
            beta_s = beta_extended[2 * dim].X
        return coeff



Auxiliary Functions

In [ ]:
def logit(y):
    y[y <= 0] = 0.001
    y[y >= 1] = 0.999
    x = np.log(y/(1-y))
    return x

def logit_inv(x):
    y = np.exp(-x)/(1+np.exp(-x))
    return y

def rmse(group):
   sites_in_group = group['target_site'].unique()
   print(sites_in_group)
   rmse_sites = []
   for site in sites_in_group:
    rez_site = group[group['target_site'] == site]
    rmse_site = np.sqrt(np.mean((rez_site['obs'] - rez_site['pred']) ** 2))
    rmse_sites.append(rmse_site)
   rmse_sites = np.array(rmse_sites)
   rmse_std = np.round(np.std(rmse_sites) * 100, decimals=2)
   rmse_mean = np.round(np.mean(rmse_sites) * 100, decimals=2)
   rmse_allsites = np.round(np.sqrt(np.mean((group['obs'] - group['pred']) ** 2)) * 100, decimals=2)
   return rmse_mean, rmse_std, rmse_allsites


# Main

In [ ]:
from scipy import linalg
import re

In [ ]:
target_sites = np.arange(1, 11, 1)#[target_site]

# Loop over target sites -----

prediction_dfs = []
for i in range(0, 1, 1): #len(target_sites), 1):

    target_site = target_sites[i]

    # A. Loop over testing months
    for testing_month_index in np.arange(1, 13, 1):
        print('target_site: ' + str(target_site) + ', test month: ' + str(testing_month_index))

        wd_train, wd_test_month = train_test_splits_windDirection(dataset_wd, testing_month_index)
        prod_train, prod_test_month = train_test_splits_Production(dataset_prod, testing_month_index)

        dataset_learning_complete = prod_train.iloc[np.isin(prod_train.index,
                                                                          wd_train.index), :]
        dataset_testing_complete = prod_test_month.iloc[np.isin(prod_test_month.index,
                                                                        wd_test_month.index), :]

        wd_train = wd_train.iloc[np.isin(wd_train.index, prod_train.index), :]
        wd_test_month = wd_test_month.iloc[np.isin(wd_test_month.index, prod_test_month.index), :]

        # B. Loop over conditioning bandwidths
        for conditioning_bandwidth in conditioning_winddirection_bandwidths:

            #print('start target site ' + str(target_site) + ', bandwidth: ' + str(conditioning_bandwidth))

            # ccf_prod_target = ccf_prod.loc[ccf_prod['site_i'] == int(target_site), :]
            # px.line(ccf_prod_target, x='lag_value', y='ccf_prod', color='site_j').show()

            # Initialize the array of Beta coefficients
            betas = np.empty((number_horizons, (number_plants * len(lags) + 1), conditioning_number_points, 1))

            # Compute weights for wind direction
            weights_train = [
                np.exp(-((wd_train[int(target_site)].values - cpoint) ** 2 / (2 * conditioning_bandwidth ** 2)))
                for cpoint in conditioning_winddirection_points_positions]
            weights_test = [
                np.exp(-((wd_test_month[int(target_site)].values - cpoint) ** 2 / (2 * conditioning_bandwidth ** 2)))
                for cpoint in conditioning_winddirection_points_positions]
            weights_test_df = pd.DataFrame(weights_test).T


            cor_prod = cor_prod[np.isin(cor_prod.site_j.astype('string'), dataset_learning_complete.columns)]

            # explanatory sites ----
            # distance cor proxy
            explanatory_sites_distProxy = cor_prod[cor_prod.site_i == int(target_site)].sort_values(by='cor', ascending=False)[:number_plants].site_j.values

            # C. Loop over horizons ---
            for h in [6]:

                #ccf_prod_lags_h = ccf_prod_target.loc[np.abs(ccf_prod_target["lag_value"]) >= h, :]
                #ccf_prod_mean_h = ccf_prod_lags_h.groupby('site_j')['ccf_prod'].mean().reset_index().sort_values(
                #    by='ccf_prod', ascending=False).iloc[:number_plants, :]
                #explanatory_sites_ccfProd = ccf_prod_mean_h['site_j'].values

                # 1. analytical solve AR and ARST \beta = (t(X)X)-1(t(X)Y)

                #print('analytical solving AR, for horizon ' +str(h) + '...')
                betas_ar, betas_ar_wide = buildAR(dataset=dataset_learning_complete, lags=lags, horizon=h,
                                                  target_site=str(target_site), plot=False)


                #print('analytical solving ARST, unweighted, for horizon ' + str(h) + '...')
                betas_arst_distProxy, betas_arst_wide_distProxy, xtx_distProxy, xty_distProxy, response_distProxy = buildARST(dataset=dataset_learning_complete,
                                                        explanatory_sites=explanatory_sites_distProxy,
                                                        lags=lags, horizon=h, nb_plants=len(explanatory_sites_distProxy),
                                                        target_site=target_site, plot=False)

                #print('analytical solving ARST, weighted, for horizon ' + str(h) + '...')
                betas_arst_weighted_distProxy, betas_arst_weighted_wide_distProxy, xtx_w_distProxy, xty_w_distProxy, response_w_distProxy= buildARSTweighted(dataset=dataset_learning_complete,
                                                                            weights=weights_train,
                                                                            conditioning_points=conditioning_winddirection_points_positions,
                                                                            conditioning_bandwidth=conditioning_bandwidth,
                                                                            explanatory_sites=explanatory_sites_distProxy,
                                                                            number_plants=len(explanatory_sites_distProxy),
                                                                            horizon=h,
                                                                            lags=lags,
                                                                            target_site=target_site, plot=False)


                #print('evaluate AR...')
                ar_scores, pred_ar = buildPrediction(betas=betas_ar, dataset=dataset_testing_complete,
                                            target_site = target_site,
                                            explanatory_sites=[target_site],
                                            horizon=h, lags=lags,
                                            number_plants=1, method_name='ar_noweights', plot=False, logit_bool=True)

                #print('RMSE AR, month ' + str(testing_month_index) + ': ' + str(ar_scores['rmse'].values))

                #print('evaluate ARST - distance proxy...')
                arst_distproxy_scores, pred_arst = buildPrediction(betas=betas_arst_distProxy, dataset=dataset_testing_complete,
                                                        target_site=target_site, explanatory_sites=explanatory_sites_distProxy,
                                                        horizon=h, lags=lags,
                                                        number_plants=number_plants, method_name='arst_distProxy_noweights',
                                                        logit_bool=True)

                #print('RMSE ARST, month ' + str(testing_month_index) + ': '  + str(arst_distproxy_scores['rmse'].values))


                arst_weighted_distproxy_scores, pred_arst_weighted = buildPrediction(betas=betas_arst_weighted_distProxy,
                                                                dataset=dataset_testing_complete,
                                                                target_site=target_site, weights=weights_test_df,
                                                                explanatory_sites=explanatory_sites_distProxy,
                                                                conditioning_bandwidth=conditioning_bandwidth,
                                                                conditioning_points=conditioning_winddirection_points_positions,
                                                                horizon=h, lags=lags,
                                                                number_plants=number_plants,
                                                                method_name='arst_distProxy_weights', logit_bool=True)

                #print('RMSE ARST - weighted, month ' + str(testing_month_index) + ': '  + str(arst_distproxy_scores['rmse'].values))


                # L1 solve per conditional weight

                rmses_list = []
                for sparsity_reduc in [2]:

                    sparsity_ratio = 1/sparsity_reduc

                    features_train = buildExplanVar(dataset=dataset_learning_complete,
                                                    lags=lags, horizon=h,
                                                    number_plants=number_plants,
                                                    explanatory_sites=explanatory_sites_distProxy)

                    features_test = buildExplanVar(dataset=dataset_testing_complete,
                                                    lags=lags, horizon=h,
                                                    number_plants=number_plants,
                                                    explanatory_sites=explanatory_sites_distProxy)

                    #print('L1 solve for sparsity reduction factor: ' + str(sparsity_reduc)  + '...')

                    weighted_lasso = True
                    sklearn_lasso = True

                    if weighted_lasso:
                        #print('solving weighted lasso...')
                        beta_list = []
                        for cpoint_index in range(len(conditioning_winddirection_points_positions)):

                            cpoint = conditioning_winddirection_points_positions[cpoint_index]
                            betas_cpoint = betas_arst_weighted_distProxy[betas_arst_weighted_distProxy['conditioning_point'] == cpoint]
                            betas_cpoint = betas_cpoint.assign(abs_beta = np.abs(betas_cpoint['beta']))
                            beta_intercept = betas_arst_weighted_distProxy[(betas_arst_weighted_distProxy['conditioning_point'] == cpoint) &
                                                  (betas_arst_weighted_distProxy['explanatory_site'] == 'intercept')]['beta'].values
                            response_cpoint = response_w_distProxy[cpoint_index]

                            if sklearn_lasso:
                              xtx_w_cpoint = xtx_w_distProxy[cpoint_index]
                              xty_w_cpoint = xty_w_distProxy[cpoint_index]

                              reg_w = Lasso(alpha=1, random_state=0, fit_intercept=False).fit(
                                X=features_train.values,
                                y=dataset_learning_complete[str(target_site)].values,
                                sample_weight=weights_train[cpoint_index])
                              betas_w = reg_w.coef_
                            else:
                              #xtx_w_cpoint = xtx_w_distProxy[cpoint_index][:-1,:-1]
                              #xty_w_cpoint = xty_w_distProxy[cpoint_index][:-1]
                              betas_cpoint = betas_cpoint[betas_cpoint['explanatory_site'] != 'intercept']# take out intercept
                              betas_sorted_cpoint = betas_cpoint.sort_values(by='abs_beta', ascending=False).reset_index(drop=True)
                              betas_threshold_cpoint = betas_sorted_cpoint.iloc[:int(np.floor(betas_sorted_cpoint.shape[0] / sparsity_reduc)),:]

                              l1_threshold = np.abs(betas_threshold_cpoint['abs_beta']).sum()

                              # derive lasso ----
                              betas_w = builARSTl1Solve_normineq(XtX=xtx_w_cpoint, XtY=xty_w_cpoint, lasso=True, l1_threshold=l1_threshold, verbose=False)
                              #betas = builARSTl1Solve_normconstraint_vRSS(X=features_train, y=dataset_learning_complete[target_site], l1_threshold=l1_threshold)



                            # retrieve results ----
                            l1_threshold_optim = np.sum(np.abs(betas_w[:-1]))
                            beta_df = pd.DataFrame({'beta': betas_w,
                                                    'conditioning_bandwidth': conditioning_bandwidth,
                                                    'conditioning_point': cpoint,
                                                    'explanatory_site': betas_arst_weighted_distProxy[betas_arst_weighted_distProxy['conditioning_point'] == cpoint]['explanatory_site'],
                                                    'explanatory_lag': betas_arst_weighted_distProxy[betas_arst_weighted_distProxy['conditioning_point'] == cpoint]['explanatory_lag'],
                                                    'sparsity_ratio': sparsity_ratio})
                            beta_list.append(beta_df)
                        betas_arst_l1 = pd.concat(beta_list)

                        arst_distProxy_w_l1_scores, pred_arst_weighted_l1 = buildPrediction(betas=betas_arst_l1,
                                                        dataset=dataset_testing_complete,
                                                        target_site=target_site, explanatory_sites=explanatory_sites_distProxy,
                                                        conditioning_bandwidth=conditioning_bandwidth,
                                                        conditioning_points=conditioning_winddirection_points_positions,
                                                        weights=weights_test_df,
                                                        horizon=h, lags=lags,
                                                        number_plants=number_plants,
                                                        method_name='arst_distProxy_w_l1', plot=False, logit_bool=True)

                        #print('RMSE ARST cwd lasso sklearn, month ' + str(testing_month_index) + ': ' + str(arst_distProxy_w_l1_scores['rmse'].values))

                    betas_arst = betas_arst_distProxy.assign(abs_beta=np.abs(betas_arst_distProxy['beta']))
                    betas_arst = betas_arst[betas_arst['explanatory_site'] != 'intercept']
                    betas_sorted_arst = betas_arst.sort_values(by='abs_beta', ascending=False).reset_index(drop=True)
                    betas_threshold_arst = betas_sorted_arst.iloc[
                                            :int(np.floor(betas_sorted_arst.shape[0] / sparsity_reduc)), :]

                    l1_threshold = np.abs(betas_threshold_arst['abs_beta']).sum()
                    betas = builARSTl1Solve_budget(XtX=xtx_distProxy, XtY=xty_distProxy, lasso=True, l1_threshold=l1_threshold, verbose=False)
                    l1_threshold_optim = np.sum(np.abs(betas[:-1]))

                    beta_df = pd.DataFrame({'beta': betas,
                                            'explanatory_site': betas_arst_distProxy['explanatory_site'],
                                            'explanatory_lag': betas_arst_distProxy['explanatory_lag'],
                                            'sparsity_ratio': sparsity_ratio})

                    arst_distProxy_l1_scores, pred_arst_l1 = buildPrediction(betas=beta_df,
                                    dataset=dataset_testing_complete,
                                    target_site=target_site, explanatory_sites=explanatory_sites_distProxy,
                                    horizon=h, lags=lags,
                                    number_plants=number_plants,
                                    method_name='arst_distProxy_l1', plot=False, logit_bool=True)

                    # build lasso from scikit learn

                    #reg = Lasso(alpha=1, random_state=0, fit_intercept=True).fit(X=features_train.iloc[:,:-1].values,
                    #                                        y=dataset_learning_complete[str(target_site)].values)
                    reg = LassoCV(alphas=alphas_lasso, random_state=0, fit_intercept=True).fit(X=features_train.iloc[:,:-1].values,
                                        y=dataset_learning_complete[str(target_site)].values)
                    reg_full = LassoCV(alphas=alphas_lasso, random_state=0, fit_intercept=False).fit(X=features_train.values,
                                                            y=dataset_learning_complete[str(target_site)].values)
                    reg_pred = logit_inv(reg.predict(X=features_test.iloc[:,:-1].values))
                    #reg_rmse = np.sqrt(np.mean((logit_inv(dataset_testing_complete[str(target_site)]) - reg_pred)**2))
                    #print('RMSE ARST lasso from sklearn, month ' + str(testing_month_index) + ': '  + str(reg_rmse))

                    betas_arst_distProxy_l1_sklrn = pd.DataFrame({'beta': np.concatenate((reg.coef_, [reg.intercept_])),
                                            'explanatory_site': betas_arst_distProxy['explanatory_site'],
                                            'explanatory_lag': betas_arst_distProxy['explanatory_lag'],
                                            'sparsity_ratio': sparsity_ratio})

                    arst_distProxy_l1_sklrn_scores, pred_arst_l1_sklrn = buildPrediction(betas=betas_arst_distProxy_l1_sklrn,
                      dataset=dataset_testing_complete,
                      target_site=target_site, explanatory_sites=explanatory_sites_distProxy,
                      horizon=h, lags=lags,
                      number_plants=number_plants,
                      method_name='arst_distProxy_l1_sklrn', plot=False, logit_bool=True)

                    pred_arst_l1_sklrn_dir = pred_arst_l1_sklrn.copy()
                    pred_arst_l1_sklrn_dir['pred'] = reg_pred
                    pred_arst_l1_sklrn_dir['method'] = 'arst_distProxy_l1_sklrn_dir'

                    rmses_list.append(arst_distProxy_l1_scores['rmse'])
                rmses = pd.concat(rmses_list)

                prediction_dfs.append(pred_ar)
                prediction_dfs.append(pred_arst)
                prediction_dfs.append(pred_arst_l1)
                prediction_dfs.append(pred_arst_l1_sklrn)
                prediction_dfs.append(pred_arst_l1_sklrn_dir)
                prediction_dfs.append(pred_arst_weighted)
                prediction_dfs.append(pred_arst_weighted_l1)
pred_testing_full = pd.concat(prediction_dfs)
print(pred_testing_full.groupby('method').apply(rmse))




target_site: 1, test month: 1
target_site: 1, test month: 2
target_site: 1, test month: 3
target_site: 1, test month: 4
target_site: 1, test month: 5
target_site: 1, test month: 6
target_site: 1, test month: 7
target_site: 1, test month: 8
target_site: 1, test month: 9
target_site: 1, test month: 10
target_site: 1, test month: 11
target_site: 1, test month: 12
['1']
['1']
['1']
['1']
['1']
['1']
['1']
method
ar_noweights                     (23.0, 0.0, 23.0)
arst_distProxy_l1              (22.09, 0.0, 22.09)
arst_distProxy_l1_sklrn        (22.08, 0.0, 22.08)
arst_distProxy_l1_sklrn_dir    (22.08, 0.0, 22.08)
arst_distProxy_noweights       (22.11, 0.0, 22.11)
arst_distProxy_w_l1            (26.61, 0.0, 26.61)
arst_distProxy_weights         (28.27, 0.0, 28.27)
dtype: object


In [ ]:

l1_nolasso = np.abs(betas_arst_distProxy['beta'].values).sum()
print(np.abs(reg.coef_).sum() / l1_nolasso)
print(np.abs(reg_full.coef_).sum() / l1_nolasso)

pred_l1_diff = pred_arst_l1_sklrn['pred'] - pred_arst_l1_sklrn_dir['pred']

0.96
0.49349537960968376
0.6312243558873352


In [ ]:
pred_nointercept = logit_inv(np.matmul(np.concatenate((reg.coef_, [reg.intercept_])), features_test.T))


pred_comparison_tbl = pd.DataFrame({'pred_dir': pred_arst_l1_sklrn_dir['pred'],
                                    'pred_analy':pred_arst_l1_sklrn['pred'],
                                    'pred_analy_no_intercept': pred_nointercept})
import plotly.express as px
px.line(pred_comparison_tbl.reset_index().melt(id_vars='TIMESTAMP'), x='TIMESTAMP', y='value', color='variable').show()

NameError: ignored